#Use a NN to model XOR

Study how individual motifs work to compose into larger circuits. Just like how studying AND gate composes into larger computer circuit, or how studying cells and particles enables understanding on a macroscopic level.

Not for a research paper, but for personal understanding, which will help for future research papers.

XOR is a geometric motif in latent space. Do other such motifs exist?

In [1]:
%%capture
import torch
import copy
import matplotlib.pyplot as plt
import numpy as np
import os 

In [17]:
class Feedforward(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Feedforward, self).__init__()
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)  # one hidden layer
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size, 1)  #output layer w/o sigmoid
        self.sigmoid = torch.nn.Sigmoid()
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        output = self.sigmoid(output)
        return output

In [18]:
model = Feedforward(2, 2)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [16]:
x_train = torch.FloatTensor(np.array([[0,0], [0,1], [1,0], [1,1]]))
y_train = torch.FloatTensor(np.array([0,1,1,0]))

torch.Size([4])

In [ ]:
model.train()
epoch = 10000

for epoch in range(epoch):
    #sets the gradients to zero before we start backpropagation. 
    #This is a necessary step as PyTorch accumulates the gradients from the backward passes from the previous epochs.
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(x_train)
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y_train)
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

In [26]:
model.eval()
y_pred = model(x_train)
y_pred

tensor([[0.0526],
        [0.9852],
        [0.9830],
        [0.0526]], grad_fn=<SigmoidBackward0>)



---
# Get activations


In [27]:
def get_activations(input, layer_name):
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    for name_to_check, layer in model.named_modules():
        if name_to_check == layer_name:
            break
    layer.register_forward_hook(get_activation(layer_name))
    
    output = model(input)

    return activation.copy()  #else will return the same actvs of model

In [63]:
# get previous last layer name
named_layers = dict(model.named_modules())
layers = list(named_layers.keys())

# too many branches, so just get the converged branch points
# '' is the entire model, so disregard it
# layers = [x for x in layers if '.' not in x and x != '']  
layers = [x for x in layers if x != '']  
layers

['fc1', 'relu', 'fc2', 'sigmoid']



---
# Compare different input activations

Compare [0, 0] and [1, 0]

In [38]:
x_train[0]

tensor([0., 0.])

In [42]:
input_1 = x_train[0]
out = model(input_1)
print(out)

tensor([0.0526], grad_fn=<SigmoidBackward0>)


In [43]:
for layer_name in layers:
    print(get_activations(input_1, layer_name))

{'': tensor([0.0526])}
{'fc1': tensor([-3.6790e-05, -2.3193e-04])}
{'relu': tensor([0., 0.])}
{'fc2': tensor([-2.8915])}
{'sigmoid': tensor([0.0526])}


In [77]:
input_2 = x_train[0] + torch.tensor([1,0])
input_2

tensor([1., 0.])

In [121]:
for layer_name in layers:
    print(get_activations(input_2, layer_name))

{'fc1': tensor([ 2.2371, -2.2851])}
{'relu': tensor([2.2371, 0.0000])}
{'fc2': tensor([4.0572])}
{'sigmoid': tensor([0.9830])}


In [122]:
for layer_name in layers:
    print( '[0,0]:', get_activations(input_1, layer_name), '[1,0]:', get_activations(input_2, layer_name))

[0,0]: {'fc1': tensor([-3.6790e-05, -2.3193e-04])} [1,0]: {'fc1': tensor([ 2.2371, -2.2851])}
[0,0]: {'relu': tensor([0., 0.])} [1,0]: {'relu': tensor([2.2371, 0.0000])}
[0,0]: {'fc2': tensor([-2.8915])} [1,0]: {'fc2': tensor([4.0572])}
[0,0]: {'sigmoid': tensor([0.0526])} [1,0]: {'sigmoid': tensor([0.9830])}


In [79]:
for layer_name in layers:
    diff = get_activations(input_2, layer_name)[layer_name] - get_activations(input_1, layer_name)[layer_name]
    print(diff)

tensor([ 2.2371, -2.2849])
tensor([2.2371, 0.0000])
tensor([6.9488])
tensor([0.9304])




---

# Why does this input change cause this difference?

Why are the activations different? Let's look at how each activation is calculated, and how the change in input causes the change in activation.

In [52]:
model

Feedforward(
  (fc1): Linear(in_features=2, out_features=2, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=2, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [40]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 2.2371, -2.2374],
        [-2.2849,  2.2847]], requires_grad=True)
Parameter containing:
tensor([-3.6790e-05, -2.3193e-04], requires_grad=True)
Parameter containing:
tensor([[3.1062, 3.1020]], requires_grad=True)
Parameter containing:
tensor([-2.8915], requires_grad=True)


In [33]:
prev_weights = model.fc1.weight.data
prev_weights

tensor([[ 2.2371, -2.2374],
        [-2.2849,  2.2847]])

In [64]:
for layer in layers:
    if isinstance(named_layers[layer], torch.nn.Linear):
        print(layer, named_layers[layer].state_dict()['bias'])

fc1 tensor([-3.6790e-05, -2.3193e-04])
fc2 tensor([-2.8915])


In [65]:
model.fc1.weight.data * input_1 + named_layers['fc1'].state_dict()['bias']

tensor([[-3.6790e-05, -2.3193e-04],
        [-3.6790e-05, -2.3193e-04]])

In [123]:
model.fc1.weight.data * input_2 + named_layers['fc1'].state_dict()['bias']

tensor([[ 2.2371e+00, -2.3193e-04],
        [-2.2849e+00, -2.3193e-04]])



---

Break down each step of W * X + b

In [69]:
model.fc1.weight.data

tensor([[ 2.2371, -2.2374],
        [-2.2849,  2.2847]])

In [70]:
input_2

tensor([0.1000, 0.0000])

In [68]:
WX_hadamard = torch.multiply(model.fc1.weight.data, input_2)
WX_hadamard

tensor([[ 0.2237, -0.0000],
        [-0.2285,  0.0000]])

WX:

[w1 w2]

[w3 w4]

w1 * (1) - w2 * 0 = w1

w3 * (1) + w4 * 0 = w3

Thus, when you change the input from [0, 0] to [1, 1], you are changing WX from [0, 0] to [w1, w3]. And we know that [1, 0] is supposed to be 1.

But we also need [1, 1] to be 0, so it's not as simple as "make it so the neuron outputs are bigger". 

Matrix multiplication is unit conversion. Row of W multiplies by column X

In [72]:
input_3 = torch.FloatTensor(np.array([1,1]))
input_3

tensor([1., 1.])

In [73]:
for layer_name in layers:
    print(get_activations(input_3, layer_name))

{'fc1': tensor([-0.0003, -0.0004])}
{'relu': tensor([0., 0.])}
{'fc2': tensor([-2.8915])}
{'sigmoid': tensor([0.0526])}


In [74]:
for layer_name in layers:
    diff = get_activations(input_3, layer_name)[layer_name] - get_activations(input_1, layer_name)[layer_name]
    print(diff)

tensor([-0.0002, -0.0002])
tensor([0., 0.])
tensor([0.])
tensor([0.])


[0, 0] and [1, 1] have the SAME neuron outputs after fc1. Let's look at the weights of fc1 again to see why that's the case.

In [75]:
model.fc1.weight.data

tensor([[ 2.2371, -2.2374],
        [-2.2849,  2.2847]])

In [76]:
WX_hadamard = torch.multiply(model.fc1.weight.data, input_3)
WX_hadamard

tensor([[ 2.2371, -2.2374],
        [-2.2849,  2.2847]])

The first element of WX is: 2.2371 -2.2374 = -0.002 

The second element is also close to 0.

So the weights were learned to make sure w1 - w2 was close to 0, as ReLU would turn that into 0.

Then in fc2 (output node), the bias is negative so that sigmoid makes it less than 0. But for [0,1] and [1,0], 

For multiple training instances, does it do this every time? There may be multiple ways for an ANN circuit to calculate XOR.







---
# Gradual input modification 

Now slightly modify the input and see what happens to each layer!

Try different modification levels. As you gradually change it, how do the weights change?


---


Use:
Yt_train = Yt_train.type(torch.LongTensor)

https://stackoverflow.com/questions/60440292/runtimeerror-expected-scalar-type-long-but-found-float

In [104]:
layer_name = 'fc1'
# step_incr = [round(x * 0.1, 1) for x in range(0, 10)]
old_input = torch.tensor([0, 0]).type(torch.LongTensor)
new_input = torch.tensor([0, 0]).type(torch.LongTensor)
# for i in step_incr:
for i in range(0, 10):
    old_input = new_input.clone().type(torch.LongTensor)
    new_input = old_input + torch.tensor([0.1, 0])
    # new_input = torch.tensor([i, 0])
    print(i)
    diff = get_activations(new_input, layer_name)[layer_name] - get_activations(old_input, layer_name)[layer_name]
    print(diff)

0


RuntimeError: ignored

In [119]:
get_activations(new_input, layer_name)[layer_name]

tensor([ 0.2237, -0.2287])

In [101]:
new_input

tensor([0.1000, 0.0000])

In [106]:
old_input = new_input.clone().type(torch.LongTensor)
old_input

tensor([0, 0])

In [118]:
new_input.dtype

torch.float32

In [116]:
torch.tensor([0, 0]).dtype

torch.int64

In [115]:
torch.tensor([0, 0]).type(torch.LongTensor).dtype

torch.int64

In [112]:
get_activations(torch.tensor([0, 0]).type(torch.LongTensor), layer_name)[layer_name]

RuntimeError: ignored



---

Conditions to allow NN to model XOR:

- the weights were learned to make sure w1 - w2 was close to 0,

Any NN with the architecture above whose weights satisfy these conditions will model XOR